In [38]:
import dask.dataframe as dd
from dask.distributed import Client
from dask_kubernetes import KubeCluster

cluster = KubeCluster(n_workers=30)
cluster

distributed.scheduler - INFO - Clear task state
/opt/conda/lib/python3.7/site-packages/distributed/dashboard/core.py:72: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn("\n" + msg)
distributed.scheduler - INFO -   Scheduler at:   tcp://10.20.0.131:46757
distributed.scheduler - INFO -   dashboard at:                    :37827


In [39]:
client = Client(cluster)

distributed.scheduler - INFO - Receive client connection: Client-87d06a62-fa5e-11e9-8129-56d0ba869408
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.151.31:44989
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.151.31:44989
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.127.33:39423
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.127.33:39423
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.123.31:40435
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.123.31:40435
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.126.33:40923
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.126.33:40923
distributed.core - INFO - Starting established connect

In [3]:
df = dd.read_parquet("gs://dask-nyc-taxi/yellowtrip.parquet", engine="fastparquet")
df

distributed.scheduler - INFO - Register tcp://10.20.123.12:42071
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.123.12:42071
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.150.12:37011
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.150.12:37011
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.135.12:33541
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.135.12:33541
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.147.12:37833
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.147.12:37833
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register tcp://10.20.144.12:39619
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.20.144.12:39619
dis

,vendor_id,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,tolls_amount,total_amount
npartitions=907,,,,,,,,,,,,,,
2014-01-01 00:00:00,category[unknown],datetime64[ns],int64,float64,float64,float64,category[unknown],float64,float64,category[unknown],float64,float64,float64,float64
2014-01-02 00:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 00:00:00,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-06-30 23:59:59,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [4]:
sdf = df.partitions[0].compute()
sdf = sdf.head(200_000)

In [5]:
sdf.head()

,vendor_id,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,tip_amount,tolls_amount,total_amount
pickup_datetime,,,,,,,,,,,,,,
2014-01-01,VTS,2014-01-01 00:50:00,1,41.62,-73.871285,40.773962,nassau,-73.618775,41.200682,CSH,179.0,0.0,5.33,185.33
2014-01-01,VTS,2014-01-01 00:18:00,1,5.58,-73.961757,40.759820,standard,-73.978477,40.743305,CSH,17.0,0.0,0.00,18.00
2014-01-01,VTS,2014-01-01 00:28:00,3,6.20,-73.956543,40.778122,standard,-74.000382,40.718385,CRD,23.0,4.7,0.00,28.70
2014-01-01,VTS,2014-01-01 00:22:00,1,7.93,-73.979027,40.749187,standard,-73.899360,40.828842,CSH,25.0,0.0,0.00,26.00
2014-01-01,VTS,2014-01-01 00:03:00,6,1.05,-73.971300,40.792705,standard,-73.959910,40.800747,CSH,5.0,0.0,0.00,6.00


In [40]:
import sklearn.base
import sklearn.ensemble
import sklearn.compose
import sklearn.pipeline
import sklearn.preprocessing

In [7]:
cat_cols = ['vendor_id', 'rate_code', 'payment_type']

y = (sdf['tip_amount'] > 0).astype(int)
X = sdf.drop("tip_amount", axis="columns")

In [64]:
import pandas as pd
import sklearn.base
    

class DayOfWeekTransformer(sklearn.base.BaseEstimator, sklearn.base.TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        result = X.copy()
        for column in X.columns:
            result[column] = X[column].dt.dayofweek
            
        return result

In [66]:
categories = [sdf[col].cat.categories for col in cat_cols]
onehot = sklearn.preprocessing.OneHotEncoder(categories=categories, sparse=False)

dayofweek = DayOfWeekTransformer()

preprocess = sklearn.compose.make_column_transformer(
    (onehot, cat_cols),
    (dayofweek, ["dropoff_datetime"]),
    remainder="passthrough"
)


pipe = sklearn.pipeline.make_pipeline(
    preprocess,
    sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=-1)
)

In [67]:
%%time
pipe.fit(X, y)

CPU times: user 9.27 s, sys: 191 ms, total: 9.46 s
Wall time: 1.76 s


Pipeline(memory=None,
         steps=[('columntransformer',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categorical_features=None,
                                                                categories=[Index(['CMT', 'VTS'], dtype='object'),
                                                                            Index(['standard', 'JFK', 'NWK', 'nassau', 'negotiated', 'group'], dtype='object'),
                                                                            Index...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                     

In [68]:
pipe = sklearn.pipeline.make_pipeline(
    preprocess,
    sklearn.ensemble.RandomForestClassifier(n_estimators=1000, n_jobs=-1)
)

In [69]:
%%time
import joblib

with joblib.parallel_backend("dask"):
    pipe.fit(X, y)

TimeoutError: DaskDistributedBackend has no worker after 10 seconds. Make sure that workers are started and can properly connect to the scheduler and increase the joblib/dask connection timeout with:

parallel_backend('dask', wait_for_workers_timeout=20)

In [43]:
cluster.scale(300)
cluster

# Distributed Grid Search

In [44]:
import sklearn.experimental.enable_hist_gradient_boosting
import dask_ml.model_selection

In [ ]:
pipe = sklearn.pipeline.make_pipeline(
    preprocess,
    sklearn.ensemble.HistGradientBoostingClassifier(n_iter_no_change=3)
)

In [46]:
param_grid = dict(
    histgradientboostingclassifier__learning_rate=[0.009, 0.01, 0.011],
    histgradientboostingclassifier__max_iter=[90, 100, 110],
    histgradientboostingclassifier__max_leaf_nodes=[25, 30, 35],
    histgradientboostingclassifier__max_depth=[3, 5, 10],
)
search = dask_ml.model_selection.GridSearchCV(pipe, param_grid, cv=3)
search

GridSearchCV(cache_cv=True, cv=3, error_score='raise',
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categorical_features=None,
                                                                                       categories=[Index(['CMT', 'VTS'], dtype='object'),
                                                                                                   Index(['standard', 'J...
                                                       

In [15]:
X = X[::4]
y = y[::4]

In [49]:
%time search.fit(X, y);

distributed.utils - ERROR - An asyncio.Future, a coroutine or an awaitable is required
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/distributed/utils.py", line 662, in log_errors
    yield
  File "/opt/conda/lib/python3.7/site-packages/distributed/dashboard/components/shared.py", line 408, in cb
    prof, metadata = await asyncio.gather(prof, metadata)
  File "/opt/conda/lib/python3.7/asyncio/tasks.py", line 719, in gather
    fut = ensure_future(arg, loop=loop)
  File "/opt/conda/lib/python3.7/asyncio/tasks.py", line 592, in ensure_future
    raise TypeError('An asyncio.Future, a coroutine or an awaitable is '
TypeError: An asyncio.Future, a coroutine or an awaitable is required
tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <zmq.eventloop.ioloop.ZMQIOLoop object at 0x7f210021e710>>, <Task finished coro=<ProfileTimePlot.trigger_update.<locals>.cb() done, defined at /opt/conda/lib

CPU times: user 56.9 s, sys: 5.97 s, total: 1min 2s
Wall time: 2min 54s


GridSearchCV(cache_cv=True, cv=3, error_score='raise',
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='passthrough',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(categorical_features=None,
                                                                                       categories=[Index(['CMT', 'VTS'], dtype='object'),
                                                                                                   Index(['standard', 'J...
                                                       

In [50]:
search.best_score_

0.57558

In [51]:
import pandas as pd

In [52]:
from IPython.display import HTML

In [54]:
cluster.scale(0)

distributed.scheduler - INFO - Remove worker tcp://10.20.139.34:33503
distributed.core - INFO - Removing comms to tcp://10.20.139.34:33503
distributed.scheduler - INFO - Remove worker tcp://10.20.123.33:45221
distributed.core - INFO - Removing comms to tcp://10.20.123.33:45221
distributed.scheduler - INFO - Remove worker tcp://10.20.149.34:40993
distributed.core - INFO - Removing comms to tcp://10.20.149.34:40993
distributed.scheduler - INFO - Remove worker tcp://10.20.151.40:35217
distributed.core - INFO - Removing comms to tcp://10.20.151.40:35217
distributed.scheduler - INFO - Remove worker tcp://10.20.144.35:44993
distributed.core - INFO - Removing comms to tcp://10.20.144.35:44993
distributed.scheduler - INFO - Remove worker tcp://10.20.153.41:36999
distributed.core - INFO - Removing comms to tcp://10.20.153.41:36999
distributed.scheduler - INFO - Remove worker tcp://10.20.130.33:39237
distributed.core - INFO - Removing comms to tcp://10.20.130.33:39237
distributed.scheduler - INF

In [28]:
search.score(X, y)

0.9971